In [2]:
import functools
import sys
import math
import scipy.io
sys.path.append("..")
import matplotlib.pyplot as plt
import numpy as np

import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()
import tensorflow_probability as tfp

import Test_Likelihood as tl
import ODE_Dynamics as od
import Positive_Symptom_fn as fn

from functools import partial

import scipy.stats as ss

import simulation as sim
import model as mdl

tfd = tfp.distributions
tfb = tfp.bijectors

In [3]:
vload = sim.sample_viral_load(mu_b = 12,sigma_b = 1)
prob_s_i0 = 0.55; prob_s_ibar0 = 0.1
p_threshold0 = 170306.4 * 1E-05
s_threshold0 = sim.get_symptom_threshold(vload)
start_day0 = 20
full_data = sim.simulate_epidemic(vload, pop_size = 10000, start_day = start_day0,
                             prob_s_i = prob_s_i0, prob_s_ibar = prob_s_ibar0, 
                             v_threshold = p_threshold0)
start_day = 0
end_day = 10
sliced_data = full_data[start_day:end_day,:]
print(sliced_data)

21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
[[ 978    1]
 [ 950    3]
 [ 971    2]
 [ 998    6]
 [ 901    3]
 [ 991    6]
 [1003    3]
 [1059    2]
 [1030    6]
 [ 960    4]]


In [4]:
# Arrange data in workable format
days = [float(i) for i in list(range(len(sliced_data)))]
tests = sliced_data[:,0]
positives = sliced_data[:,1]
test_data = np.column_stack((days, tests, positives))
test_data = tf.cast(test_data, dtype = tf.float32)
print(test_data)

tf.Tensor(
[[0.000e+00 9.780e+02 1.000e+00]
 [1.000e+00 9.500e+02 3.000e+00]
 [2.000e+00 9.710e+02 2.000e+00]
 [3.000e+00 9.980e+02 6.000e+00]
 [4.000e+00 9.010e+02 3.000e+00]
 [5.000e+00 9.910e+02 6.000e+00]
 [6.000e+00 1.003e+03 3.000e+00]
 [7.000e+00 1.059e+03 2.000e+00]
 [8.000e+00 1.030e+03 6.000e+00]
 [9.000e+00 9.600e+02 4.000e+00]], shape=(10, 3), dtype=float32)


In [5]:
vdyn_ode_fn = od.ViralDynamics
positive_fn = fn.proba_pos_sym(p_threshold0).positive_fn
symptom_fn = fn.proba_pos_sym(p_threshold0).symptom_fn
prob_s_ibar = prob_s_ibar0
sample_size = 1000
k = 1
index = 1
mu_b, sigma_b = 12, 1
beta = np.random.normal(mu_b, sigma_b, 1)   #"rate at which virus infects host cells"
L = 0.0025/beta

V0 = np.random.normal(1E3, 1E2, 1)
X0 = 1E6
Y0 = V0

par=np.array([[L,0.01,beta*1E-7,0.5,20.0,10.0,V0,X0,Y0]])

init_state=(np.array([[V0,X0,Y0]], dtype=np.float32))

while index <= sample_size - 1:
    beta = np.random.normal(mu_b, sigma_b, 1)   #"rate at which virus infects host cells"
    L = 0.0025/beta
    
    V0 = np.random.normal(1E3, 1E2, 1)
    X0 = 1E6
    Y0 = V0
    
    par_new=np.array([[L,0.01,beta*1E-7,0.5,20.0,10.0,V0,X0,Y0]])
    par = np.concatenate((par, par_new), axis = 0)
    
    init_state_new=(np.array([[V0,X0,Y0]], dtype=np.float32))
    init_state = np.concatenate((init_state, init_state_new), 0)

    index +=1
        

vpar = tf.constant(par, dtype=tf.float32)
pospar = par
sympar = par
print(par.shape)

(1000, 9)


In [6]:
# Joint prior on nu, i, s
# nu ~ Beta(2,8), so that mode = 0.1
# i, s, (1-i-s) ~ Dirichlet(0.1,0.1,0.1)

joint_prior = tfd.JointDistributionNamed(dict(
    recov_rate = tfd.Beta(2,8),
    epi_state = tfd.Dirichlet(
    concentration = np.ones(3, np.float32)/10),
))

In [7]:
# Joint log prior
def joint_log_prior(R0, nu, epi_state):
    epi_state = np.array(epi_state)
    return joint_prior.log_prob(
      recov_rate = nu, epi_state = epi_state)- math.log(R0)

In [8]:
# Joint log-likelihood
# Prior on R0: 1/R0

def joint_log_prob(test_data, R0, nu, epi_state):
    epi_state = np.array(epi_state)
    i = epi_state[...,0]
    s = epi_state[...,1]
    epipar = tf.constant(np.array([[R0, 5.0E-08, nu, i, s]], dtype=np.float32))

    loglike = mdl.loglik(test_data, vdyn_ode_fn, positive_fn, symptom_fn, prob_s_ibar = 0.1, prob_fp=0.0, Epi_Model=od.SIR,
                 duration= start_day0, Epi_cadence = 0.5, Vir_cadence = 0.0675, phi_s = 0.55, psi_s = 0.1)
    ll,pp = loglike.__call__(test_data, epipar, vpar, pospar, sympar)
    epi_state = [i,s,1.0-i-s]
    ll = ll + joint_log_prior(R0, nu, epi_state).numpy()
    return sum(ll)


In [9]:
joint_log_prob(test_data, R0 = 1.8, nu = 0.1, epi_state = [0.01,0.9, 0.09])

initial time
tf.Tensor(-21.0, shape=(), dtype=float32)
final time
tf.Tensor(10.0, shape=(), dtype=float32)


<tf.Tensor: shape=(), dtype=float32, numpy=-4256.0967>

In [10]:
# Compute log_lik 
# Impute things we don't want to sample, eg. observations

unnormalized_posterior_log_prob = partial(joint_log_prob, test_data)


In [11]:
# Initialize parameters

R0 = 1.8; nu0 = 0.1; i0 = 0.001; s0 = 0.99; r0 = 1.0 - i0 -s0;
initial_state = [R0, nu0, [i0, s0, r0]]
initial_state

[1.8, 0.1, [0.001, 0.99, 0.009000000000000008]]

In [12]:
# Transform all parameters so that parameter spaces are unconstrained
# Every parameter needs a bijector, might be 'Identity'

unconstraining_bijectors = [
    tfb.Log(),
    tfb.Invert(tfb.Sigmoid()),
    tfb.Invert(tfb.IteratedSigmoidCentered())
]

In [13]:
n_sample = 50
n_burnin = 30
n_adaptation = 24

In [14]:
#@tf.function(autograph=False)
def sample():
    return tfp.mcmc.sample_chain(
        num_results = n_sample,
        num_burnin_steps = n_burnin,
        current_state = initial_state,
        kernel = tfp.mcmc.SimpleStepSizeAdaptation(
            tfp.mcmc.TransformedTransitionKernel(
                inner_kernel = tfp.mcmc.HamiltonianMonteCarlo(
                    target_log_prob_fn = unnormalized_posterior_log_prob,
                     step_size = 0.01,
                     num_leapfrog_steps = 2),
                bijector = unconstraining_bijectors),
             num_adaptation_steps = n_adaptation),
        trace_fn=lambda _, pkr: pkr.inner_results.inner_results.is_accepted)

In [15]:
[R0, nu, epi_state], is_accepted = sample()

initial time
tf.Tensor(-21.0, shape=(), dtype=float32)
final time
tf.Tensor(10.0, shape=(), dtype=float32)
initial time
tf.Tensor(-21.0, shape=(), dtype=float32)
final time
tf.Tensor(10.0, shape=(), dtype=float32)
initial time
tf.Tensor(-21.0, shape=(), dtype=float32)
final time
tf.Tensor(10.0, shape=(), dtype=float32)
initial time
tf.Tensor(-21.0, shape=(), dtype=float32)
final time
tf.Tensor(10.0, shape=(), dtype=float32)
initial time
tf.Tensor(-21.0, shape=(), dtype=float32)
final time
tf.Tensor(10.0, shape=(), dtype=float32)
initial time
tf.Tensor(-21.0, shape=(), dtype=float32)
final time
tf.Tensor(10.0, shape=(), dtype=float32)


ValueError: The two structures don't have the same nested structure.

First structure: type=list str=[TensorSpec(shape=(), dtype=tf.int32, name=None), TensorSpec(shape=(2,), dtype=tf.int32, name=None), [TensorSpec(shape=(), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None), [TensorSpec(shape=(), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None)]], SimpleStepSizeAdaptationResults(
  inner_results=TransformedTransitionKernelResults(
      transformed_state=[TensorSpec(shape=(), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None), TensorSpec(shape=(4,), dtype=tf.float32, name=None)],
      inner_results=MetropolisHastingsKernelResults(
          accepted_results=UncalibratedHamiltonianMonteCarloKernelResults(
              log_acceptance_correction=TensorSpec(shape=(), dtype=tf.float32, name=None),
              target_log_prob=TensorSpec(shape=(), dty

In [ ]:
# Compute posterior means of Epidemic parameters

acceptance_rate = tf.reduce_mean(tf.cast(is_accepted, dtype=tf.float32)).numpy()
mean_R0 = tf.reduce_mean(R0, axis=0).numpy()
mean_nu = tf.reduce_mean(nu, axis=0).numpy()
i_s = np.array(i_s)
i = i_s[...,0]
s = i_s[...,0]
mean_i = tf.reduce_mean(i, axis=0).numpy()
mean_s = tf.reduce_mean(s, axis=0).numpy()


In [ ]:
print('acceptance_rate:', acceptance_rate)
print('avg basic reproduction number:', mean_R0)
print('avg recovery rate:', mean_nu)
print('avg infected', mean_i)
print('avg susceptible', mean_s)

In [ ]:
# Forecast Infection Rate, 7 Days into Future
forecast_window = 7
R0_sample = R0[n_burnin:n_sample]
nu_sample = nu[n_burnin:n_sample]
i_sample = i[n_burnin:n_sample]
s_sample = s[n_burnin:n_sample]

mod = od.SIR(par)

par=tf.constant(np.array([[R0_sample[1].numpy(), 5.0E-08, nu_sample[1].numpy()]], dtype=np.float32))
init_state=tf.constant(np.array([[i_sample[1].numpy(),s_sample[1].numpy()]], dtype=np.float32))
init_time=tf.constant(0.0)
int_duration = end_day - start_day + forecast_window
soln_times=tf.constant(np.linspace(0.0,int_duration,num=int_duration,dtype=np.float32))
dp = tfp.math.ode.DormandPrince()
results = dp.solve(mod.RHS, init_time, init_state, solution_times=soln_times)
i_forecast = results.states[:,0,0].numpy()


In [ ]:
id = 1
while id < len(R0_sample):
    par=tf.constant(np.array([[R0_sample[id].numpy(), 5.0E-08, nu_sample[id].numpy()]], dtype=np.float32))
    init_state=tf.constant(np.array([[i_sample[id].numpy(),s_sample[id].numpy()]], dtype=np.float32))
    init_time=tf.constant(0.0)
    soln_times=tf.constant(np.linspace(0.0,int_duration,num=int_duration,dtype=np.float32))
    dp = tfp.math.ode.DormandPrince()
    results = dp.solve(mod.RHS, init_time, init_state, solution_times=soln_times)
    i_forecast = np.column_stack((i_forecast, results.states[:,0,0].numpy()))
    id += 1


In [ ]:
i_forecast_lower = []
i_forecast_upper = []
i_forecast_median = []

for j in range(forecast_window):
    id = end_day - start_day + j
    i_forecast_lower.append(np.percentile(i_forecast[id], 5))
    i_forecast_upper.append(np.percentile(i_forecast[id], 95))
    i_forecast_median.append(np.percentile(i_forecast[id], 50))
    
print(i_forecast_lower)
print(i_forecast_upper)
print(i_forecast_median)

In [ ]:
par=tf.constant(np.array([[1.8, 5.0E-08,0.1]], dtype=np.float32))
mod = od.SIR(par)
init_state=tf.constant(np.array([[0.001,0.999]], dtype=np.float32))
init_time=tf.constant(0.0)
soln_times=tf.constant(np.linspace(0.0,100.0,num=100,dtype=np.float32))
dp = tfp.math.ode.DormandPrince()
results = dp.solve(mod.RHS, init_time, init_state, solution_times=soln_times)
i_true = results.states[:,0,0].numpy()
t1 = end_day
t2 = end_day + 7
i_true = i_true[t1:t2]
print(i_true)

In [ ]:
import matplotlib.pyplot as plt
t = list(range(len(i_forecast_median)))
plt.plot(t, i_forecast_lower, "b--",label = "95-th percentile")
plt.plot(t, i_forecast_upper,"b--",label = "5-th percentile")
plt.plot(t, i_forecast_median, "b-",label = "Forecasted Median")
plt.plot(t, i_true, "r-", label = "True Infected")
plt.title('Forecast Confidence Interval')
plt.legend(loc="upper right")
plt.savefig('Forecast.png', dpi=300, bbox_inches='tight')

In [ ]:
# Staircase Plot

import matplotlib.pyplot as plt
import staircase as sc
inverse_data = np.column_stack((R0_sample.numpy(), nu_sample.numpy(), i_sample.numpy(), s_sample.numpy()))
figname = "mcmc_staircase.png"
ndim = 4
scales = [0,0,0,0]
scales[0] = (min(R0_sample),max(R0_sample))
scales[1] = (min(nu_sample),max(nu_sample))
scales[2] = (min(i_sample),max(i_sample))
scales[3] = (min(s_sample),max(s_sample))

bin_num = 10
axlabel = ["R0", "nu", "i", "s"]
#sc.staircase_plot(inverse_data, figname, hist_scalex, hist_scaley, bin_num, xtrue = [1,1,-2,-1,1])
sc.staircase_plot_simple(inverse_data, figname, ndim, scales, bin_num, axlabel)



In [ ]:
# Plot MCMC evolution

import matplotlib.pyplot as plt
fig, axs = plt.subplots(2, 2)
t = list(range(len(s)))
axs[0, 0].plot(t, R0.numpy(), "b-")
axs[0, 0].set_title('R0')
axs[0, 1].plot(t, nu.numpy(), "b-")
axs[0, 1].set_title('nu')
axs[1, 0].plot(t, i.numpy(), "b-")
axs[1, 0].set_title('i')
axs[1, 1].plot(t, s.numpy(), "b-")
axs[1, 1].set_title('s')

for ax in axs.flat:
    ax.set(xlabel='MCMC draws', ylabel='Time')

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()
    
plt.savefig('MCMC_runs.png', dpi=300, bbox_inches='tight')